In [4]:
 # %pip install robin_stocks openai==1.2.2 yfinance requests_oauthlib twilio boto3 --upgrade 

In [2]:
import robin_stocks.robinhood as r
import requests
import json
import pandas as pd
import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta
from dateutil.tz import tzlocal
from datetime import timedelta
import pytz
import boto3
from tqdm.auto import tqdm

In [3]:
from news2roi import News2ROI

In [4]:
s3 = boto3.client('s3')

In [5]:
bucket_name = 'jvalansi-notebooks-data'

In [6]:
# s3.upload_file("/root/.aws/secrets.json", bucket_name, ".aws.json")

In [7]:
# s3.upload_file("/root/.rapidapi.json", bucket_name, ".rapidapi.json")

In [8]:
# s3.upload_file("/root/.newsapi.json", bucket_name, ".newsapi.json")

In [9]:
data = s3.get_object(Bucket=bucket_name, Key=".aws.json")
j = json.loads(data['Body'].read().decode('utf8'))
openai_key = j['OPENAI_KEY']
data = s3.get_object(Bucket=bucket_name, Key=".rapidapi.json")
# data = s3.get_object(Bucket=bucket_name, Key=".newsapi.json")
j = json.loads(data['Body'].read().decode('utf8'))
news_key = j["key"]


In [10]:
# s3.upload_file("/root/.robinhood.json", bucket_name, ".robinhood.json")

In [11]:
data = s3.get_object(Bucket=bucket_name, Key=".robinhood.json")
trade_cred = json.loads(data['Body'].read().decode('utf8'))

In [12]:
data = s3.get_object(Bucket=bucket_name, Key=".twilio.json")
notify_cred = json.loads(data['Body'].read().decode('utf8'))

In [13]:
n2r = News2ROI(openai_key, news_key, trade_cred, notify_cred)

In [13]:
date = datetime.datetime.today().strftime('%Y-%m-%d')

In [ ]:
# date = "2024-01-26"

In [ ]:
# date = "2024-01-16"

In [14]:
# get new news articles
news = n2r.get_news(date=date)

In [15]:
articles = news #['articles']

In [16]:
articles = [article for article in articles if News2ROI.contains_words(article, words={'United States',"US"})]

In [17]:
len(articles)

6

In [18]:
articles_ = [article for article in articles if n2r.is_recent(article['publishedAt']['date'], delta=datetime.timedelta(hours=24))]

In [19]:
len(articles_)

6

In [20]:
# get sentiment
res = []


In [21]:
for article in tqdm(articles):
    res += [n2r.analyse_article(article)]
    

  0%|          | 0/6 [00:00<?, ?it/s]

In [22]:
candidates = News2ROI.get_candidates(res)

In [23]:
candidates

,stock,ticker,exchange,reasoning,action,date,data
1,Julius Baer Group,JBAXY,OTC Markets,The announcement of a significant writeoff of ...,3.0,2024-01-28 14:03:44.000000,Julius Baer set to announce writeoff linked to...


In [36]:
# ticker = low_sentiment.iloc[0]['ticker']

In [54]:
candidate = candidates.iloc[0].copy(deep=True)

In [55]:
ticker = candidate['ticker']

In [14]:
ticker = 'aapl'

In [15]:
# get options


In [16]:
# expirationDate = the next friday

In [17]:
option_data = n2r.get_option_data(ticker)

Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data 

In [18]:
len(option_data)

58

In [20]:
option_df = News2ROI.parse_option_data(option_data)

In [125]:
# option_df.plot(x='strike_price', y='adjusted_mark_price')

In [124]:
# option_df.plot(x='strike_price', y='break_even_price')

In [122]:
# find lowest option price with highest break even price
# usually the current stock price
# example stock costs 100
# option to put (sell) in a week for 100 costs 1$
# if the stock costs 99 in a week, I break even, anything below I earn
# e.g if the stock costs 98 in a week, I sell at 100, I get 2$, but it cost 1$, I made 1$

# if I buy the option to sell in 150, it costs 49.85
# if the stock costs 100 in a week, I sell at 150, I made 50, but it cost me 49.85, I made 0.15$

# if I buy the option to sell in 90, it costs 0.02
# if the stock costs 100 in a week, I sell in 90, I lost 10$

# options are at a multiples of 100s
# so if the stock costs 100
# and costs 4 to sell at 104
# if the stock goes to 101, it'll cost 3, so I could sell at 3$ and lose 100
# I could lose everythin
# If it goes to 99, it'll cost 5, and I could sell at 5, and gain 100
# I could 

In [74]:
# I want to maximize ROI
# I need to check which one maximizes for the expected value of 0.2% drop

In [22]:
row = option_df[option_df['normalized_gain']==option_df['normalized_gain'].max()].iloc[0].to_dict()

In [31]:
candidate = pd.Series()

In [34]:
candidate['strike_price'] = row['strike_price']

candidate['bid_price'] = row['bid_price']

candidate['current_price'] = row['current_price']

In [28]:
# when the target price hits, the option prices should shift with the target price
# say current price is 100

In [ ]:
# maybe easier to calculate - each price shifts one up

In [ ]:
# place option for the day
r.order_buy_option_limit

In [ ]:
# place reminder to sell tomorrow


In [ ]:
# sell previous reminders


In [35]:
print(candidate.to_json(indent=2))

{
  "strike_price":182.5,
  "bid_price":0.55,
  "current_price":192.45
}


In [146]:
# j = {'account_sid': 'AC417be496c2958add9742abcc83840cb7',
#      'auth_token': '32e6cee11411c6037525b250c973eb2f',
#      'from': '+18662593587',
#      'to': '+14159002444'}


In [110]:
# with open('/root/.twilio.json', 'w') as f:
#     f.write(json.dumps(j))

In [111]:
# s3.upload_file("/root/.twilio.json", bucket_name, ".twilio.json")

In [36]:
n2r.notify(candidate)

'SM3f71b689a0218ebf167e9eece4f288eb'